# Smart Planning : appreciation model (v1) - Neural nets with scikit

 
### Details 

- Problem : `Regression`


- Models :
    * scikit Neural network MLP
    
    
- label : numeric in the interval [0,10]


- Values in percentage to avoid the problem related to a specefic number of classrooms, teachers ...


<br>

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics, model_selection
import math
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
%matplotlib inline

# Get the dataset

In [10]:
df = pd.read_excel("dataset2.xlsx")
df.head()

,SC_simultanite_Perc,salles_utilisees_Perc,salles_surutiliseess_Perc,places_videss_Perc,seances_samedis_Perc,Smidis_Perc,IntvDep8Hs_Perc,InterDepDisp6s_Perc,GrpDep8Hs_Perc,GrpDepDisp6s_Perc,SHDispoIntrv_Nbr,SDepDIntvs_Perc,SHPDIntvs_Perc,Label
0,4,0,0,0,2,4,6,2,5,20,1,1,0,9
1,13,10,20,25,20,2,25,6,2,8,10,30,10,7
2,17,100,100,28,0,6,26,6,3,22,0,20,33,8
3,16,70,30,0,1,2,26,10,3,0,4,5,3,8
4,17,12,20,3,0,3,26,8,2,0,22,1,3,10


<br>

# Insights about the dataset

> To get insight : [click here](./DataSetStudy.ipynb)

<br>

# Splitting the dataset

In [11]:

# few data 
trainning_set = df.sample(frac=0.75, random_state=25) 
test_set = df.drop(trainning_set.index)

""" # without normalizing data 

# Calculate the Z-scores of each column in the training set:
trainning_set_mean = trainning_set.mean()
trainning_set_std = trainning_set.std()
trainning_set = (trainning_set - trainning_set_mean)/trainning_set_std

# Calculate the Z-scores of each column in the testing set:
test_set_mean = test_set.mean()
test_set_std = test_set.std()
test_set = (test_set - test_set_mean)/test_set_std


"""

print("# DataSet shape : ", df.shape)
print("# trainning_set shape : ", trainning_set.shape)
print("# test_set shape  20% : ", test_set.shape)


X = trainning_set.iloc[:,0:13]
Y = trainning_set[['Label']]


X_test = test_set.iloc[:,0:13]
Y_test = test_set[['Label']]


# DataSet shape :  (521, 14)
# trainning_set shape :  (391, 14)
# test_set shape  20% :  (130, 14)


<br>

# Create & trainning the model


In [12]:

# here he is, 
# check hyperParams : https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn-neural-network-mlpregressor
MLP_Regression = MLPRegressor(random_state=1, max_iter=500, hidden_layer_sizes=( 100,300,600,1000,600,300,200 ), activation='relu' ) 
MLP_Regression.fit( X, Y.values.ravel() )

# print model parameters
print("# best_loss_ : \n", MLP_Regression.best_loss_ )
#print("\n # coefs_ : ", MLP_Regression.coefs_ )

X_test = test_set.iloc[:,0:13]
Y_test =  test_set[['Label']]
Y_pred = MLP_Regression.predict(X_test)
  
# Calculate metrics using test set
MLP_Regression_score = round( MLP_Regression.score(X_test, Y_test.values.ravel() ), 2)
MLP_Regression_mae = round(metrics.mean_absolute_error(Y_test, Y_pred), 4)
MLP_Regression_mse = round(metrics.mean_squared_error(Y_test, Y_pred), 4)


# best_loss_ : 
 0.007816977728474016


## testing set with model predictions

In [13]:

testing_set_with_predictions = test_set.copy()
testing_set_with_predictions["MLP_Regression_model_predictions"] = MLP_Regression.predict(test_set.iloc[:,0:13])
testing_set_with_predictions.head()


,SC_simultanite_Perc,salles_utilisees_Perc,salles_surutiliseess_Perc,places_videss_Perc,seances_samedis_Perc,Smidis_Perc,IntvDep8Hs_Perc,InterDepDisp6s_Perc,GrpDep8Hs_Perc,GrpDepDisp6s_Perc,SHDispoIntrv_Nbr,SDepDIntvs_Perc,SHPDIntvs_Perc,Label,MLP_Regression_model_predictions
2,17,100,100,28,0,6,26,6,3,22,0,20,33,8,9.545396
5,0,70,0,0,0,0,0,0,0,0,0,0,0,10,10.503588
14,100,100,100,90,100,100,100,100,100,100,15,100,100,0,0.395675
17,10,70,21,7,22,76,12,15,13,14,2,15,22,6,4.842013
22,17,77,10,13,0,0,22,18,15,14,13,11,10,9,9.120984


<br>
<hr>
<br>

# Results & Comparaison

In [14]:
pd.DataFrame({  
        'Model' : [ 
            'MLP_Regression',
        ],
        'Score' : [ 
            MLP_Regression_score,
        ],
        'MAE'  : [ 
            MLP_Regression_mae,
        ],
        'MSE'  : [ 
            MLP_Regression_mse,
        ]
    })

,Model,Score,MAE,MSE
0,MLP_Regression,0.22,1.6376,5.4201


<br>

## manual testing

In [22]:
seance_en_simultanite_perc = 10    # Pourcentage de séances avec contrainte de simultanéité
salles_utilisees_perc = 82         # Pourcentage de salles utilisées
salles_surutiliseess_Perc = 10     # Pourcentage de salles surutilisées (44Heures /Semaine)
places_videss_Perc = 10            #  Pourcentage de places vides
seances_samedis_Perc = 0           # Pourcentage de séances de samedi
Smidis_Perc = 100                    # Pourcentage de séances de midi 
IntvDep8Hs_Perc = 0                # Pourcentage d'intervenants qui depassent 8Heures /Jour
InterDepDisp6s_Perc = 0            # Pourcentage d'intervenants qui depassent le dispatchement 6Heures/Jour
GrpDep8Hs_Perc = 0                 # Pourcentage de groupes qui depassent 8Heures /Jour
GrpDepDisp6s_Perc = 0              # Pourcentage de groupes qui depassent le dispatchement 6Heures/Jour
SHDispoIntrv_Nbr = 0               # Pourcentage de séances qui dépassent la disponibilité des intrervenants
SDepDIntvs_Perc = 0                # Pourcentage de séances hors préférences de disponibilités des intervenants
SHPDIntvs_Perc = 0                 # nombre de seances hors disponibilités d'intervenant

manual_testing_var = [ seance_en_simultanite_perc, salles_utilisees_perc, salles_surutiliseess_Perc, places_videss_Perc, seances_samedis_Perc, 
Smidis_Perc, IntvDep8Hs_Perc, InterDepDisp6s_Perc, GrpDep8Hs_Perc, GrpDepDisp6s_Perc, SHDispoIntrv_Nbr, SDepDIntvs_Perc, SHPDIntvs_Perc]

In [23]:

pd.DataFrame({  
        'Model' : [ 
            'MLP_Regression Model',  
        ],
        'Score' : [ 
            MLP_Regression.predict([ manual_testing_var ])[0],
        ]})


,Model,Score
0,MLP_Regression Model,7.55378
